In [2]:
# --- TB DETECTION APP (RUN THIS CELL) ---
import numpy as np
import cv2
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# 1. LOAD THE SAVED MODEL
# This loads the "brain" you already trained, so you don't have to train again.
try:
    model = tf.keras.models.load_model('tb_detection_hybrid_model.h5')
    print("✅ Model loaded successfully!")
except:
    print("❌ Error: 'tb_detection_hybrid_model.h5' not found. Make sure it is in this folder.")

# 2. DEFINE THE FILTERS (Essential for the Hybrid approach)
# [cite_start]These process the image exactly how the model expects[cite: 21, 191].
def apply_gabor(image):
    g_kernel = cv2.getGaborKernel((31, 31), 4.0, 0, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    return cv2.filter2D(image, cv2.CV_8UC3, g_kernel)

def apply_canny(image):
    return cv2.Canny(image, 100, 200)

# 3. CREATE THE UPLOAD BUTTON WIDGET
uploader = widgets.FileUpload(accept='.jpg, .png, .jpeg', multiple=False)
output_box = widgets.Output()

display(widgets.Label("Upload a Chest X-ray to start diagnosis:"))
display(uploader)
display(output_box)

def on_upload_change(change):
    with output_box:
        clear_output()
        print("Processing image...")
        
        try:
            # Get the uploaded file
            if not uploader.value: return
            
            # Extract content (handling different Jupyter versions)
            vals = uploader.value
            content = vals[0]['content'] if isinstance(vals, tuple) else list(vals.values())[0]['content']
            
            # Decode Image
            nparr = np.frombuffer(content, np.uint8)
            img_array = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
            
            # Pre-process & Filter
            resized_img = cv2.resize(img_array, (224, 224))
            gabor = apply_gabor(resized_img)
            canny = apply_canny(resized_img)
            combined_img = np.dstack((resized_img, gabor, canny))
            
            # Normalize & Predict
            final_input = combined_img.astype('float32') / 255.0
            final_input = np.expand_dims(final_input, axis=0)
            
            prediction_score = model.predict(final_input)[0][0]
            
            # Show Result
            if prediction_score > 0.5:
                result_text = f"TUBERCULOSIS DETECTED\nConfidence: {prediction_score*100:.2f}%"
                color = 'red'
            else:
                result_text = f"NORMAL (Healthy)\nConfidence: {(1-prediction_score)*100:.2f}%"
                color = 'green'

            plt.figure(figsize=(6, 6))
            plt.imshow(resized_img, cmap='gray')
            plt.title(result_text, color=color, fontweight='bold', fontsize=14)
            plt.axis('off')
            plt.show()
            
        except Exception as e:
            print(f"Error: {e}")

uploader.observe(on_upload_change, names='value')

✅ Model loaded successfully!


Label(value='Upload a Chest X-ray to start diagnosis:')

FileUpload(value=(), accept='.jpg, .png, .jpeg', description='Upload')

Output()